# 🤗 Verfahren

## Welche Verfahren sind geeignet?

Alle QA-Modelle basieren auf vortrainierten Sprachmodellen, die allermeisten auf Transformer-Architekturen (v.a. BERT). Die Basismodelle sind auf allgemeinen Task trainiert (zum Beispiel Masking, Next Token Prediction). Der Vorteil ist, dass dieses Training auf riesigen nicht annotierten Datenmengen erfolgen kann. Dieses Verfahren kann man als unsupervised bezeichnen, da die Daten nicht annotiert sind, bzw. die nötige Annotation (maskieren von Tokens) automatisiert erfolgen kann.

Ein Finetuning auf einem Domain spezifischen Datensatz (als Vorstufe zum Finetuning auf einen Domain spezifischen Task, wie QA) ist denkbar aber anscheinend nicht die Regel [1].

Grundsätzlich suchen wir nach Modellen für Extractive QA, wobei das Modell eine Antwort aus einen gegebenen Kontext extrahiert (verglichen etwa mit Open Generative QA, wobei eine Freitext-Antwort generiert wird) [2]. 

Die Allermeisten Extractive-QA-Modelle sind so trainiert, dass die Prediction des Modells bei einer gegeben Frage + Kontext zwei Wahrscheinlichkeiten ausgibt: Eine für den Start der Antwort Sequenz und eine für das Ende. Die Textpassage zwischen den beiden Tokens mit der höchsten Wahrscheinlichkeit ist dann die Antwort. Im Falle, dass der End Token vor dem Anfangs Token auftritt, ist die Antwort ein leerer String [3].

[1] https://discuss.huggingface.co/t/fine-tuning-bert-model-on-domain-specific-language-and-for-classification/3106/6 \
[2] https://huggingface.co/tasks/question-answering \
[3] https://blog.paperspace.com/how-to-train-question-answering-machine-learning-models/

## Vortrainiertes Netz ausreichend / Domänenspezifisches Training notwendig?

Das Basis NLP-Modell wird (aufgabenspezifisch) ge-finetuned an Hand eines - für den Ziel Task annotierten - Datensatzes. Für QA-Modelle ist das meistens SQuAD oder SQuAD 2 [1]. Es gibt eine Vielzahl an NLP-Modellen, die auf diesen Task an Hand einer der beiden Datensätze ge-finetuned sind, da dieser jedoch in Englisch ist, sind diese für deutsches QA nicht geeignet. 

Es gibt jedoch auch einen GermanQaAD [2] und dazugehörige finetuned Modelle [3]. Manchen von diesen sind allerdings auch auf deQuAD trainiert, einer (automatischen) Übersetzung von SuAD ins Deutsche. 

Für die meisten QA-Task reichen die vortrainierten Modelle aus (die auf eher generischen Daten trainiert wurden, wie Wikipedia-Artikel zu verschiedensten Themen). Für stark spezifische Domains empfiehlt sich jedoch entweder ein bereits auf SQuAD (oder GermanQaAD) getuntes Modell noch weiter auf einem annotierten, domainspezifischen Datensatz zu tunen, oder (wenn der eigene Datensatz groß genug ist) ein Basismodell (wie BERT) auf den spezifischen QA-Task zu tunen. Als Größenordnung sind hier ~2000 benötigte Samples zu finden [4]. Ein How-To ist hier [5] zu finden. 

Wenn man ausgehend von einem NLP-Basismodell Task spezifisch finetuned, dann ist die Definition des Task offen, das heißt zum Beispiel ein reines Zuordnen einer Frage zu einem Dokument in der Knowledge Base wäre möglich (verglichen mit dem - durch vortrainierten Modelle gegeben - Task der Ausgabe der genauen Anfangs- und End Tokens der Antwort Passage). 

Das Finetuning ist (im Vergleich zum Training des Basismodells) also supervised. Ein unsupervised Finetuning (Finetuning eines QA-Netzes ohne annotierte Daten) ist nicht möglich.

![Mögliche Workflows für die Nutzung / Training von QA-Modellen](model_workflows.png)
*Mögliche Workflows für die Nutzung / Training von QA-Modellen* 


[1] https://rajpurkar.github.io/SQuAD-explorer/ \
[2] https://arxiv.org/pdf/2104.12741.pdf \
[3] https://huggingface.co/models?language=de&pipeline_tag=question-answering&sort=downloads \
[4] (https://haystack.deepset.ai/docs/tutorial2md \
[5] https://huggingface.co/transformers/v3.2.0/custom_datasets.html#question-answering-with-squad-2-0




# 💾 Daten

## Ground Truth / Wie müssen die Daten gelabelt sein?

Als Ground Truth bietet sich ein Datensatz an, der eine Reihe von Fragen mit den dazugehörigen Text Passagen aus Doku beinhaltet (dem Format von SQuAD folgend). Dabei werden mehrere Fragen pro Kontext formuliert, und mehrere Antworten auf jede Frage annotiert [1] (Das gilt für das Validation Dataset, bei dem auch Fragen ohne Antworten vorkommen können (negative samples - Verhältnis ~50/50); für das Trainings Set gibt es jedoch nur positive samples und eine Antwort pro Frage [2]).

Gegenüber diesem können die Verfahren verglichen werden. 

Dieser Datensatz muss händisch annotiert werden. Dafür würde sich das Annotation Tool von Haystack anbieten [3]. Der Umfang hängt davon ab, ob der Datensatz als reiner Testdatensatz oder zum Finetuning des Modells verwendet werden soll. Als Annotations Referenz kann der von Haystack veröffentlichte Leitfaden verwendet werden [4].

Bezüglich des Umfangs lassen sich keine genauen Angaben finden, letztendlich soll er die Breite aller möglichen Fragen abfassen [5].

[1] https://huggingface.co/datasets/squad/viewer/plain_text/validation \
[2] https://huggingface.co/datasets/squad \
[3] https://github.com/deepset-ai/haystack/tree/master/annotation_tool \
[4] https://haystack.deepset.ai/guides/annotation \
[5] https://qa.fastforwardlabs.com/domain%20adaptation/transfer%20learning/specialized%20datasets/qa/medical%20qa/2020/07/22/QA-for-Specialized-Data.html

## Welche Daten sind notwendig (pro Verfahren) / Welche Features brauchen wir?

Für alle Verfahren reicht der Textkörper aus. Je nach Verfahren sind weitere Preprocessing Schritte notwendig, wie etwa padding eines Satzes auf eine fixe Länge, wofür sowohl PyTorch [?] als auch die Huggingface API [?] Funktionen zur Verfügung stellen.

Für die Anwendung selbst müssen im Preprocessing jedoch noch weitere Informationen gespeichert werden, wie etwa 

Pfad des html-Dokuments
Link zum Paragraf im html-Dokument

um die gefundene Antwort in der Dokumentation ausfindig machen zu können. 

Das hängt schließlich davon ab, was als Dokument begriffen wird

- html-Datei (1 Seite der Doku)
- Unterabschnitt einer Seite (div / p / ul / li)

## Wie werden die Features repräsentiert?

Features könnten als Spalten einer Tabelle (PandasDataFrame / csv) repräsentiert werden. 

# 🧮 Vergleich von Modellen / Baseline Modelle / Metriken

## Mögliche Baseline Modelle

### Stichwortsuche

### Bag of Words (?)

### Word2Vec

[1] https://towardsdatascience.com/calculating-document-similarities-using-bert-and-other-models-b2c1a29c9630

### Sentence Similarity

[1] https://huggingface.co/tasks/sentence-similarity

### Passage Ranking

Passage Ranking Modelle nehmen ein Set an Dokumenten und eine Anfrage Query und geben Ranking der Dokumente heraus, wie gut die Query auf alle verfügbaren Dokumente passt [1]. Das könnte man als simples QA Modell verwenden, indem die Query die Frage und die Dokumente die Knowledge Base darstellen. 

Passage Ranking bietet sich auch Vorverarbeitungsschritt einer Anfrage an ein QA-Modell an. So kann Inferenzzeit gespart werden, wenn verfügbare Dokumente in der Knowledge Base erst nach ihrer Relevanz sortiert und dann die Antwort nur im relevantesten (bzw in den n meist relevanten) gesucht wird [2].

[1] https://huggingface.co/amberoad/bert-multilingual-passage-reranking-msmarco \
[2] https://huggingface.co/tasks/question-answering

## quantitative Gütemaße

### Exact Match (EM)

Für jedes Frage / Antwort Paar wird die Prediction gegenüber der Ground Truth verglichen. Nur bei einer genauen Überlappung der Text Passage wird die Prediction als “Treffer” gezählt. Verschiebung um eine Position im Satz, macht die Prediction zu einem Miss.

### F1-Score
Hier zählt die Schnittmenge von Prediction und wahrer Antwort. Precision ist dabei die Anzahl der Wörter in der Schnittmenge gegenüber aller Wörter in der Prediction, Recall ist das Verhältnis der Wörter in der Schnittmenge zu allen Wörtern in der Ground Truth. 

![equation f1 score](f1_score.png)

### Eigene

Je nach Task können eigene Metriken definiert werden: Ist die Aufgabe etwa eine Frage nur einem Abschnitt zuzuordnen, ist die genaue Positionierung der Satztokens jedoch irrelevant, könnte eine Accuracy angegeben werden, in wie vielen Fällen die Zuordnung zum Richtigen Dokument erfolgte. 

## Public Benchmarks /  Welche gibt es Speziell im NLP-Bereich?

QA-Modelle werden auf einer Reihe von Datasets gebenchmarkt, vorne dran SQuAD 1 & SQuAD 2, sowie die jeweiligen dev Splits (Validierungsdaten mit negativen Samples und Mehrfachantworten) [1].

[2] zeigt etwa eine Übersicht der Performance der besten Modelle auf dem SQuAD 2 Datensatz.  

Die Relevanz für unser Problem ist aber ein bisschen fraglich, denn die Modelle müssen zum einen vortrainiert verfügbar sein, zum anderen auf deutschen Datensätzen finetuned sein. 

[1] https://paperswithcode.com/task/question-answering \
[2] https://paperswithcode.com/sota/question-answering-on-squad20


# ❓Fragen Backlog

### Qualitativer Verfahrensvergleich auf Problemstellung bezogen

### Quantitativer, grundsätzlicher Verfahrensvergleich mit Benchmarks
